In [1]:
%pylab tk

import numpy as np
import mahotas as mh
import os
import cv2
import json
from time import time
from math import radians
from skimage import io, color, img_as_float, img_as_ubyte
from skimage.transform import rescale
from skimage.util import pad
from IPython.parallel import Client
from sklearn.decomposition import PCA, RandomizedPCA
from scipy.cluster.vq import kmeans2
from sklearn.cluster import DBSCAN
from sklearn.svm import SVC
from matplotlib.patches import Rectangle

import helpers.features as fh
import helpers.display as dh

BASE_PATH = os.getcwd()
print("Current base path: {0}".format(BASE_PATH))
DATA_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_cut_2/'
TO_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_outlined/'
TEST_FILES = json.loads(open(BASE_PATH + '/Daten/2D/testfiles.json').read())

MIN_PARTITION_SIZE = 100
MIN_PARTIION_WIDTH = 10
MAX_POSSIBLE_SPLIT_POINTS = 20
REQUIRED_DIFFERENCE_TO_SPLIT = 0.2
FEATURE_FN = mh.features.pftas

def find_split_point_y(image, window):
    if (window[1]-window[0]) * (window[3] - window[2]) <= MIN_PARTITION_SIZE:
        return None
    if window[3] - window[2] < MIN_PARTIION_WIDTH:
        raise Exception("Smaller than Partition Width got through")
    
    #print("Window", window)
    
    possible_split_points = range(window[2]+1, window[3]-1)
    if len(possible_split_points) > MAX_POSSIBLE_SPLIT_POINTS:
        possible_split_points = []
        split_len = int(abs(window[3] - window[2]) / MAX_POSSIBLE_SPLIT_POINTS)
        for i in range(1, MAX_POSSIBLE_SPLIT_POINTS):
            possible_split_points.append(window[2] + i * split_len)
    possible_split_points = filter(lambda x: x - window[2] > MIN_PARTIION_WIDTH and window[3] - x > MIN_PARTIION_WIDTH, possible_split_points)
    #print("Possible Splits", possible_split_points)
    
    if len(possible_split_points) > 0:
        distances = []
        for i in possible_split_points:
            dist = np.linalg.norm(FEATURE_FN(image[window[0]:window[1], window[2]:i, :]) - FEATURE_FN(image[window[0]:window[1], i+1:window[3], :]))
            distances.append(dist)
        distances = np.array(distances)
        #print("Distances", distances)
    
        possible_split_point = np.argmax(distances)
        if distances[possible_split_point] > REQUIRED_DIFFERENCE_TO_SPLIT:
            print("Split {0}".format(possible_split_points[possible_split_point]))
            return possible_split_points[possible_split_point]
    return None

def find_split_point_x(image, window):
    if (window[1]-window[0]) * (window[3] - window[2]) <= MIN_PARTITION_SIZE:
        return None
    if window[1] - window[0] < MIN_PARTIION_WIDTH:
        raise Exception("Smaller than Partition Width got through")
    
    #print("Window", window)
    
    possible_split_points = range(window[0]+1, window[1]-1)
    if len(possible_split_points) > MAX_POSSIBLE_SPLIT_POINTS:
        possible_split_points = []
        split_len = int(abs(window[1] - window[0]) / MAX_POSSIBLE_SPLIT_POINTS)
        for i in range(1, MAX_POSSIBLE_SPLIT_POINTS):
            possible_split_points.append(window[0] + i * split_len)
    possible_split_points = filter(lambda x: x - window[0] > MIN_PARTIION_WIDTH and window[1] - x > MIN_PARTIION_WIDTH, possible_split_points)
    #print("Possible Splits", possible_split_points)
    
    if len(possible_split_points) > 0:
        distances = []
        for i in possible_split_points:
            dist = np.linalg.norm(FEATURE_FN(image[window[0]:i, window[2]:window[3], :]) - FEATURE_FN(image[i+1:window[1], window[2]:window[3], :]))
            distances.append(dist)
        distances = np.array(distances)
        #print("Distances", distances)
    
        possible_split_point = np.argmax(distances)
        if distances[possible_split_point] > REQUIRED_DIFFERENCE_TO_SPLIT:
            print("Split {0}".format(possible_split_points[possible_split_point]))
            return possible_split_points[possible_split_point]
    return None

def find_split_point(image, window, dimension):
    if dimension == 0:
        return find_split_point_y(image, window)
    elif dimension == 1:
        return find_split_point_x(image, window)
    else:
        raise Exception("Unkown Dimension {0}".format(dimension))
        

def do_split(image, partition, dimension, split_value):
    rect = partition['rect']
    
    if dimension == 1:
        rect1 = np.array([rect[0], split_value, rect[2], rect[3]])
        rect2 = np.array([split_value+1, rect[1], rect[2], rect[3]])
    elif dimension == 0:
        rect1 = np.array([rect[0], rect[1], rect[2], split_value])
        rect2 = np.array([rect[0], rect[1], split_value+1, rect[3]])
    else:
        raise Exception("Unkown Dimension {0}".format(dimension))
    
    return (
        {
            'rect': rect1,
            'features': FEATURE_FN(image[rect1[0]:rect1[1], rect1[2]:rect1[3]]).flatten(),
            'last_dim': dimension
        },
        {
            'rect': rect2,
            'features': FEATURE_FN(image[rect2[0]:rect2[1], rect2[2]:rect2[3]]).flatten(),
            'last_dim': dimension
        }
    )

def extract_partitions(image):
    num_dimensions = 2
    partitions = [
        {
            'rect': np.array([ 0, image.shape[0], 0, image.shape[1] ]),
            'features': FEATURE_FN(image).flatten(),
            'last_dim': 0
        }
    ]
    finished_partitions = []
    while len(partitions) != 0:
        partition = partitions.pop()
        window = partition['rect']
        features = partition['features']
        dimension = (partition['last_dim'] + 1) % num_dimensions
        
        split_value = find_split_point(image, window, dimension)
        if split_value:
            part1, part2 = do_split(image, partition, dimension, split_value)
            partitions.append(part1)
            partitions.append(part2)
        else:
            finished_partitions.append(partition)
    return finished_partitions

def append_locality(features):
    x = np.tile(np.arange(NUMBER_OF_SECTIONS) + 1, NUMBER_OF_SECTIONS)
    y = np.repeat(np.arange(NUMBER_OF_SECTIONS) + 1, NUMBER_OF_SECTIONS)
    locality = np.divide(np.array([ x, y ]).transpose(), float(NUMBER_OF_SECTIONS))
    
    return np.concatenate((features, locality), axis=1)

def do_clustering(image, features):
    #db = DBSCAN(eps=0.3, min_samples=25).fit(features)
    #labels = db.labels_
    centroids, labels = kmeans2(features, 4)
    return labels.reshape((image.shape[0], image.shape[1]))
    
def reduce_feature_complexity(features):
    pca = PCA(n_components=6)
    #pca = RandomizedPCA(n_components=10)
    reduced = pca.fit_transform(features)
    return reduced

Populating the interactive namespace from numpy and matplotlib
Current base path: C:\Users\Stefan\Dropbox\Masterarbeit


In [2]:
file = TEST_FILES[2]
rgb_image = io.imread(DATA_PATH + file['filename'])

partitions = extract_partitions(rgb_image)

Split 91
Split 1155
Split 1726
Split 1175
Split 1772
Split 1464
Split 1811
Split 1477
Split 1513
Split 1787
Split 1544
Split 1800
Split 1800
Split 1525
Split 1533
Split 1536
Split 1784
Split 1501
Split 1799
Split 1800
Split 1476
Split 1476
Split 1501
Split 1805
Split 1442
Split 1820
Split 1820
Split 1423
Split 1423
Split 1190
Split 1789
Split 1204
Split 1204
Split 1791
Split 1230
Split 1739
Split 1359
Split 1752
Split 1378
Split 1378
Split 1753
Split 1345
Split 1327
Split 1247
Split 1749
Split 1194
Split 1761
Split 1206
Split 1214
Split 1761
Split 1214
Split 1738
Split 1738
Split 1195
Split 1194
Split 1782
Split 1194
Split 173
Split 1321
Split 251
Split 1333
Split 1639
Split 1345
Split 1676
Split 1366
Split 1689
Split 1538
Split 1709
Split 1709
Split 1479
Split 1423
Split 1538
Split 1689
Split 1376
Split 1658
Split 1393
Split 1521
Split 1651
Split 1358
Split 1665
Split 1664
Split 1365
Split 1712
Split 1411
Split 1563
Split 1538
Split 1576
Split 1576
Split 1424
Split 1592
Split 1510
Spl

In [3]:
fig, axes = plt.subplots()
axes.imshow(rgb_image)
for partition in partitions:
    rect = partition['rect']
    axes.add_patch(Rectangle((rect[2], rect[0]), rect[3] - rect[2], rect[1]-rect[0], fill=None))
fig.canvas.render()

AttributeError: 'FigureCanvasTkAgg' object has no attribute 'render'